## NPS and Wait Time Pulled from 2024

In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('AWT vs. NPS v3.csv')

# Convert 'CREATED_AT' to datetime, handling mixed formats
df['CREATED_AT'] = pd.to_datetime(df['CREATED_AT'], format='mixed', errors='coerce')

# Drop rows with invalid dates (optional, based on your needs)
df = df.dropna(subset=['CREATED_AT'])

# Create a new column for the hour of the appointment
df['hour'] = df['CREATED_AT'].dt.floor('H')

# Extract the date for the "Date" column
df['Date'] = df['hour'].dt.date

# Classify NPS scores into Promoters, Passives, and Detractors
df['promoter'] = df['NET_PROMOTER_SCORE'].apply(lambda x: 1 if 9 <= x <= 10 else 0)
df['detractor'] = df['NET_PROMOTER_SCORE'].apply(lambda x: 1 if 0 <= x <= 6 else 0)

# Group by hour and calculate required columns
hourly_nps = df.groupby('hour').agg(
    Date=('Date', 'first'),  # Extract the date for the period
    WAIT=('WAIT_TIME', 'mean'),  # Average wait time
    promoters_count=('promoter', 'sum'),
    detractors_count=('detractor', 'sum'),
    total_count=('NET_PROMOTER_SCORE', 'count')
)

# Calculate %Promoters and %Detractors
hourly_nps['%promoters'] = hourly_nps['promoters_count'] / hourly_nps['total_count'] * 100
hourly_nps['%detractors'] = hourly_nps['detractors_count'] / hourly_nps['total_count'] * 100

# Calculate NPS
hourly_nps['NPS'] = hourly_nps['%promoters'] - hourly_nps['%detractors']

# Select and rename columns
hourly_nps = hourly_nps[['Date', 'WAIT', 'NPS']]

# Save the result to a new CSV file
hourly_nps.to_csv('Hourly_NPS_with_WAIT.csv', index=False)

print(hourly_nps)


                           Date       WAIT        NPS
hour                                                 
2024-11-26 00:00:00  2024-11-26  84.802703  87.770270
2024-11-26 01:00:00  2024-11-26  77.094702  87.019868
2024-11-26 02:00:00  2024-11-26  94.356798  85.438233
2024-11-26 03:00:00  2024-11-26  79.499346  89.856021
2024-11-26 04:00:00  2024-11-26  85.056116  87.578815
2024-11-26 05:00:00  2024-11-26  81.707105  86.863271
2024-11-26 06:00:00  2024-11-26  77.283702  86.250838
2024-11-26 07:00:00  2024-11-26  79.265190  87.594937
2024-11-26 08:00:00  2024-11-26  90.742414  86.850978
2024-11-26 09:00:00  2024-11-26  82.971237  88.227425
2024-11-26 10:00:00  2024-11-26  88.349120  87.415426
2024-11-26 11:00:00  2024-11-26  87.173543  86.574984
2024-11-26 12:00:00  2024-11-26  83.797836  84.719405
2024-11-26 13:00:00  2024-11-26  83.446213  84.213877
2024-11-26 14:00:00  2024-11-26  83.689044  87.840405
2024-11-26 15:00:00  2024-11-26  82.956815  89.001350
2024-11-26 16:00:00  2024-11

/tmp/ipykernel_7404/3028278122.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['hour'] = df['CREATED_AT'].dt.floor('H')
